In [613]:
!pip install torch_geometric

In [614]:
import torch
import torch.nn as nn
from torch.nn import Linear, Parameter
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader
from torch_geometric.nn import GATConv, MessagePassing
from torch_geometric.transforms import ToSparseTensor
from torch_geometric.utils import add_self_loops, degree
from torch_geometric.datasets import TUDataset, Planetoid
from sklearn.model_selection import train_test_split
from torch_geometric.nn import global_mean_pool
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

In [615]:
class CustomGraphDataset(torch.utils.data.Dataset):
    def __init__(self, mutag, protein):
        self.mutag = TUDataset(root='/tmp/MUTAG', name='MUTAG') if mutag else None
        self.protein = TUDataset(root='/tmp/PROTEINS', name='PROTEINS') if protein else None

    def __len__(self):
        return len(self.mutag) if self.mutag is not None else len(self.protein)

    def __getitem__(self, idx):
        if self.mutag is not None:
            return self.mutag[idx]
        elif self.protein is not None:
            return self.protein[idx]


# To use the data loader, split the data into training and validation sets,
# and create data loaders for the two datasets separately.

dataset = CustomGraphDataset(mutag=True, protein=True)

# Perform a train/test split
train_dataset, test_dataset = train_test_split(dataset, test_size=0.2, random_state=42)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32)

/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:22: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [616]:
for batch in train_loader:
    print(batch.edge_attr.shape)
    break

torch.Size([1328, 4])


In [617]:
class EGConv(MessagePassing):
    def __init__(self, node_feature_dim, edge_feature_dim, hidden_dim):
        super(EGConv, self).__init__(aggr='add')

        self.edge_transform = Linear(edge_feature_dim, hidden_dim)

        # Add dropout in the edge_cnn layers
        self.edge_cnn = nn.Sequential(
            nn.ReLU(),
            nn.Dropout(0.5), # Add dropout layer
            Linear(hidden_dim, hidden_dim)
        )

        # Add a dropout to the input features before the GAT layer
        self.dropout = nn.Dropout(0.5)

        # Increase the number of heads in the GATConv. This increases model capacity
        self.gat = GATConv(
            in_channels=node_feature_dim,
            out_channels=hidden_dim,
            heads=8 # Increase number of heads
        )

    def forward(self, x, edge_index, edge_attr, batch):
        edge_attr = self.edge_transform(edge_attr)
        edge_index, _ = add_self_loops(edge_index, num_nodes=x.size(0))

        x = self.dropout(x) # Apply dropout to input features
        graph_features = self.gat(x, edge_index)

        graph_features = global_mean_pool(graph_features, batch)

        return graph_features

In [618]:
node_feature_dim = 7 # node features
edge_feature_dim = 4 # edge features
hidden_dim = 128
model = EGConv(node_feature_dim, edge_feature_dim, hidden_dim)


# Loss function and optimizer
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.70001 + 1e-4)

# Create a DataLoader for your dataset
# Create a DataLoader for your dataset
dataset = CustomGraphDataset(mutag=True, protein=True)
loader = DataLoader(dataset, batch_size=32, shuffle=True)

# Training loop
num_epochs = 100
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=30, gamma=0.01)

for epoch in range(num_epochs):
    model.train()
    for data in loader:
        optimizer.zero_grad()
        out = model(data.x, data.edge_index, data.edge_attr, data.batch)
        loss = criterion(out, data.y)
        loss.backward()
        optimizer.step()
        scheduler.step()

    # Print or log the training loss at the end of each epoch
    print(f"Epoch {epoch + 1}, Loss: {loss.item()}")

# Evaluation (you may need a separate validation dataset)
model.eval()

Epoch 1, Loss: 1.345521330833435
Epoch 2, Loss: 0.8116307258605957
Epoch 3, Loss: 0.7747025489807129
Epoch 4, Loss: 0.5745782852172852
Epoch 5, Loss: 0.4808226227760315
Epoch 6, Loss: 0.7200015187263489
Epoch 7, Loss: 0.65891033411026
Epoch 8, Loss: 0.6042097210884094
Epoch 9, Loss: 0.630759596824646
Epoch 10, Loss: 0.5626121759414673
Epoch 11, Loss: 0.47192808985710144
Epoch 12, Loss: 0.4269729554653168
Epoch 13, Loss: 0.5770348906517029
Epoch 14, Loss: 0.46360188722610474
Epoch 15, Loss: 0.5588556528091431
Epoch 16, Loss: 0.5402517318725586
Epoch 17, Loss: 0.4709661602973938
Epoch 18, Loss: 0.5681589245796204
Epoch 19, Loss: 0.5578107237815857
Epoch 20, Loss: 0.6604284644126892
Epoch 21, Loss: 0.5950865149497986
Epoch 22, Loss: 0.5922209620475769
Epoch 23, Loss: 0.4948349893093109
Epoch 24, Loss: 0.5926079154014587
Epoch 25, Loss: 0.5412247180938721
Epoch 26, Loss: 0.556644856929779
Epoch 27, Loss: 0.44538140296936035
Epoch 28, Loss: 0.7246737480163574
Epoch 29, Loss: 0.5707387328147

EGConv()

In [619]:
# Initialize a list to hold all predictions and true labels
all_preds = []
all_labels = []

for data in test_loader:
    # Pass each batch through the model
    out = model(data.x, data.edge_index, data.edge_attr, data.batch)

    # Get the prediction
    # Since your task is multi-class, choose the class with the highest output as the prediction
    pred = out.argmax(dim=1)

    # Save prediction and labels
    all_preds.extend(pred.tolist())
    all_labels.extend(data.y.tolist())

# Convert to lists
all_preds = [int(i) for i in all_preds]
all_labels = [int(i) for i in all_labels]

# Calculate metrics
accuracy = accuracy_score(all_labels, all_preds)
precision = precision_score(all_labels, all_preds, average='weighted')  # set average='weighted' to calculate for multi-class/multi-label problems
recall = recall_score(all_labels, all_preds, average='weighted')  # set average='weighted' to calculate for multi-class/multi-label problems
f1 = f1_score(all_labels, all_preds, average='weighted')  # set average='weighted' to calculate for multi-class/multi-label problems

print('Accuracy: ', accuracy)
print('Precision: ', precision)
print('Recall: ', recall)
print('F1 Score: ', f1)

Accuracy:  0.7631578947368421
Precision:  0.7844393592677346
Recall:  0.7631578947368421
F1 Score:  0.7690655209452202
